In [ ]:
pip install openai pandas tqdm


In [ ]:
# ✅ Print actual columns
print("🔍 Columns in CSV:", df.columns.tolist())

# ✅ Auto-detect column name that might contain tweets
possible_tweet_columns = ["tweet", "text", "Tweet", "message", "content"]
tweet_col = None
for col in df.columns:
    if col.strip() in possible_tweet_columns:
        tweet_col = col
        break

if not tweet_col:
    raise ValueError("CSV must contain a column with tweets. Expected one of: 'tweet', 'text', 'message', etc.")

print(f"✅ Found tweet column: {tweet_col}")
print(df[tweet_col].head())


NameError: name 'df' is not defined

In [ ]:
import openai

# ✅ Direct API key setup (only for private use)
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# OR use environment method if key is set using os.environ before
# openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
import openai
print(openai.__version__)


0.28.0


In [ ]:
pip install openai==0.28

In [ ]:
import openai
import pandas as pd
from tqdm import tqdm
import zipfile
import os

# -------------------------------
# ✅ Step 1: Setup OpenAI API Key
# -------------------------------
# Replace with your actual key — keep it secret!
openai.api_key = "sk-your_actual_openai_api_key_here"

# -------------------------------
# ✅ Step 2: Load and Unzip Dataset
# -------------------------------
zip_path = "/content/Sentiment Analysis Dataset export 2025-06-05 15-15-19.zip"

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

# Detect CSV file
csv_path = None
for filename in os.listdir("/content/"):
    if filename.endswith(".csv"):
        csv_path = os.path.join("/content/", filename)
        break

if not csv_path:
    raise FileNotFoundError("❌ No CSV file found inside the ZIP archive.")

# Load CSV
df = pd.read_csv(csv_path)

# -------------------------------
# ✅ Step 3: Detect Tweet Column
# -------------------------------
print("🔍 Columns in CSV:", df.columns.tolist())

possible_tweet_columns = ["tweet", "text", "Tweet", "message", "content"]
tweet_col = None
for col in df.columns:
    if col.strip() in possible_tweet_columns:
        tweet_col = col
        break

if not tweet_col:
    raise ValueError("❌ CSV must contain a column with tweets. Expected one of: 'tweet', 'text', 'message', etc.")

print(f"✅ Found tweet column: '{tweet_col}'")
print("🔹 Sample Tweets:")
print(df[tweet_col].dropna().head())

# -----------------------------------------
# ✅ Step 4: Define LLM Sentiment Analyzer
# -----------------------------------------
def analyze_sentiment_llm(tweet):
    try:
        tweet = str(tweet).strip()
        if not tweet:
            return "Neutral"

        prompt = f"""You are a sentiment analysis expert. Classify the sentiment of the following tweet using only one word: Positive, Negative, or Neutral.

Tweet: "{tweet}"
Sentiment:"""

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # You can switch to "gpt-4" if you have access
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0
        )
        result = response['choices'][0]['message']['content'].strip().capitalize()

        # Ensure clean and valid response
        if result not in ["Positive", "Negative", "Neutral"]:
            return "Uncertain"
        return result
    except Exception as e:
        print(f"⚠️ Error processing tweet:\n{tweet}\nException: {e}")
        return "Error"

# ------------------------------------------------
# ✅ Step 5: Run Sentiment Analysis with Progress
# ------------------------------------------------
tqdm.pandas()
df["sentiment"] = df[tweet_col].progress_apply(analyze_sentiment_llm)

# ----------------------------------------
# ✅ Step 6: Save Results to New CSV File
# ----------------------------------------
output_path = "/content/teen_tweet_sentiment_results.csv"
df.to_csv(output_path, index=False)
print(f"✅ Sentiment analysis complete and saved to: {output_path}")


🔍 Columns in CSV: ['Date', 'App', 'Usage (minutes)', 'Notifications', 'Times Opened']


ValueError: ❌ CSV must contain a column with tweets. Expected one of: 'tweet', 'text', 'message', etc.

In [ ]:
import openai
import pandas as pd
from tqdm import tqdm
import zipfile
import os

# -------------------------------
# ✅ Step 1: Setup OpenAI API Key
# -------------------------------
openai.api_key = os.getenv("OPENAI_API_KEY")  # or replace with: openai.api_key = "your-api-key-here"

# -------------------------------
# ✅ Step 2: Load and Unzip Dataset
# -------------------------------
zip_path = "/content/Sentiment Analysis Dataset export 2025-06-05 15-15-19.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

# Detect CSV file
csv_path = None
for filename in os.listdir("/content/"):
    if filename.endswith(".csv"):
        csv_path = os.path.join("/content/", filename)
        break

if not csv_path:
    raise FileNotFoundError("❌ No CSV file found inside the ZIP archive.")

df = pd.read_csv(csv_path)

# -------------------------------
# ✅ Step 3: Detect Tweet Column
# -------------------------------
print("🔍 Columns in CSV:", df.columns.tolist())

possible_tweet_columns = ["tweet", "text", "Tweet", "message", "content"]
tweet_col = None
for col in df.columns:
    if col.strip() in possible_tweet_columns:
        tweet_col = col
        break

if not tweet_col:
    raise ValueError("❌ CSV must contain a column with tweets. Expected one of: 'tweet', 'text', 'message', etc.")

print(f"✅ Found tweet column: '{tweet_col}'")
print("🔹 Sample Tweets:")
print(df[tweet_col].head())

# -----------------------------------------
# ✅ Step 4: Define LLM Sentiment Analyzer
# -----------------------------------------
def analyze_sentiment_llm(tweet):
    try:
        prompt = f"""
You are a sentiment analysis expert. Classify the sentiment of the following tweet using only one word: Positive, Negative, or Neutral.

Tweet: "{tweet}"
Sentiment:"""

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # or use "gpt-4" if available
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0
        )
        result = response['choices'][0]['message']['content'].strip()
        return result
    except Exception as e:
        print(f"⚠️ Error processing tweet: {tweet}\nException: {e}")
        return "Error"

# ------------------------------------------------
# ✅ Step 5: Run Sentiment Analysis with Progress
# ------------------------------------------------
tqdm.pandas()
df["sentiment"] = df[tweet_col].progress_apply(analyze_sentiment_llm)

# ----------------------------------------
# ✅ Step 6: Save Results to New CSV File
# ----------------------------------------
output_path = "/content/teen_tweet_sentiment_results.csv"
df.to_csv(output_path, index=False)
print(f"✅ Sentiment analysis complete and saved to: {output_path}")


🔍 Columns in CSV: ['Date', 'App', 'Usage (minutes)', 'Notifications', 'Times Opened']


ValueError: ❌ CSV must contain a column with tweets. Expected one of: 'tweet', 'text', 'message', etc.

In [ ]:
print("🔍 Available columns in CSV:", df.columns.tolist())

# Manually specify tweet column if auto-detect fails
tweet_col = None
possible_tweet_columns = ["tweet", "text", "Tweet", "message", "content"]
for col in df.columns:
    if col.strip().lower() in [c.lower() for c in possible_tweet_columns]:
        tweet_col = col
        break

# 🔧 Manually assign if still not found
if not tweet_col:
    print("⚠️ Auto-detection failed. Please assign tweet_col manually.")
    tweet_col = "your_actual_column_name_here"  # 🔁 Replace with real column name from print above

# Verify
print(f"✅ Using tweet column: {tweet_col}")


🔍 Available columns in CSV: ['Date', 'App', 'Usage (minutes)', 'Notifications', 'Times Opened']
⚠️ Auto-detection failed. Please assign tweet_col manually.
✅ Using tweet column: your_actual_column_name_here


In [ ]:
# Auto-fix: Pick longest string column as tweet_col
if not tweet_col:
    string_cols = df.select_dtypes(include='object')
    tweet_col = string_cols.apply(lambda col: col.str.len().mean()).idxmax()
    print(f"⚠️ Auto-selected '{tweet_col}' as tweet column based on text length.")


In [ ]:
# Detect Tweet Column
print("🔍 Columns in CSV:", df.columns.tolist())

tweet_col = None
possible_tweet_columns = ["tweet", "text", "Tweet", "message", "content"]
for col in df.columns:
    if col.strip().lower() in [c.lower() for c in possible_tweet_columns]:
        tweet_col = col
        break

if not tweet_col:
    print("⚠️ Auto-detect failed. Trying fallback method...")
    try:
        string_cols = df.select_dtypes(include='object')
        tweet_col = string_cols.apply(lambda col: col.str.len().mean()).idxmax()
        print(f"✅ Auto-selected tweet column: '{tweet_col}'")
    except Exception as e:
        raise ValueError("❌ Could not find a valid tweet column. Please check your CSV file.") from e

print(f"🔹 Sample Tweets:\n{df[tweet_col].head()}")


🔍 Columns in CSV: ['Date', 'App', 'Usage (minutes)', 'Notifications', 'Times Opened']
⚠️ Auto-detect failed. Trying fallback method...
✅ Auto-selected tweet column: 'Date'
🔹 Sample Tweets:
0    2024-08-07
1    2024-08-08
2    2024-08-26
3    2024-08-22
4    2024-08-12
Name: Date, dtype: object


In [ ]:
!pip install python-dotenv


In [ ]:
from dotenv import load_dotenv


In [ ]:
!pip install python-dotenv langchain openai pandas tqdm


In [ ]:
!pip install langchain-community langchain-core langchain openai python-dotenv pandas tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
pip install langchain-community


In [ ]:
!pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.43
    Uninstalling langsmith-0.3.43:
      Successfully uninstalled langsmith-0.3.43
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [ ]:
from langchain.chat_models import ChatOpenAI


In [ ]:
from langchain.chat_models import ChatOpenAI


In [ ]:
!pip install -U langchain langchain-openai python-dotenv


In [ ]:
!pip install --upgrade openai


In [ ]:
print("OPENAI_API_KEY:", os.getenv("OPENAI_API_KEY"))


OPENAI_API_KEY: None


In [ ]:
import os
import re
import json
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import openai
from tqdm import tqdm

# --- Load Keys ---
load_dotenv()
print("OPENAI_API_KEY:", os.getenv("OPENAI_API_KEY"))

# Initialize LangChain Chat Model (no openai_api_key param needed)
llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)


# --- WhatsApp Chat Extraction ---
def extract_whatsapp_messages(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat = f.read()
    # Regex to capture WhatsApp message lines: date, time, sender, message
    pattern = re.compile(r'^\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} [APMapm]{2} - ([^:]+): (.+)$')
    messages = []
    for line in chat.split("\n"):
        match = pattern.match(line)
        if match:
            sender, msg = match.groups()
            if msg.strip() and "media omitted" not in msg.lower():
                messages.append(f"{sender}: {msg}")
    return messages


def analyze_chat(messages, n=50):
    recent = "\n".join(messages[-n:])
    prompt_template = """
You are a futuristic AI therapist from 2030.

Analyze these WhatsApp messages:
- Emotional tone (stress, joy, anxiety)
- Mental clarity & decision style
- Mindset type: proactive, reactive, balanced

Recommend:
- 3 apps/habits to avoid
- 3 uplifting movies and songs
- 3 good daily mental health habits

Output ONLY in JSON format as:
{{"emotional_tone": "...", "clarity": "...", "mindset": "...", "avoid": [...], "recommend": {{"movies": [...], "songs": [...]}}, "habits": [...] }}

Chat:
{chat}
"""
    prompt = PromptTemplate(input_variables=["chat"], template=prompt_template)
    formatted_prompt = prompt.format(chat=recent)

    response = llm(formatted_prompt)  # <-- fixed here

    try:
        result = json.loads(response)
        return result
    except json.JSONDecodeError:
        print("⚠️ Failed to parse JSON from chat analysis, raw output:")
        print(response)
        return response


# --- Screen Time Analysis ---
def load_screen_time(csv_path):
    return pd.read_csv(csv_path)


def analyze_screen_time(df):
    readable = df.to_string(index=False)
    prompt_template = """
You are a digital wellness AI.

Analyze this screen time data:
- Focus vs distraction
- Burnout, overuse, addiction
- Decision fatigue signs

Recommend:
- Mental clarity (0-100)
- Avoid apps
- 3 inspiring movies and calming songs
- 3 digital detox habits

Output ONLY in JSON format as:
{{"clarity_score": 0-100, "fatigue": "...", "avoid_apps": [...], "recommend": {{"movies": [...], "songs": [...]}}, "habits": [...] }}

Screen Time Data:
{data}
"""
    prompt = PromptTemplate(input_variables=["data"], template=prompt_template)
    formatted_prompt = prompt.format(data=readable)

    response = llm(formatted_prompt)  # <-- fixed here

    try:
        result = json.loads(response)
        return result
    except json.JSONDecodeError:
        print("⚠️ Failed to parse JSON from screen time analysis, raw output:")
        print(response)
        return response


# --- Twitter Sentiment Analysis ---
def analyze_tweets(df):
    print("🔍 Columns in CSV:", df.columns.tolist())
    tweet_col = None
    for col in df.columns:
        if col.strip().lower() in ["tweet", "text", "message", "content"]:
            tweet_col = col
            break
    if not tweet_col:
        string_cols = df.select_dtypes(include='object')
        tweet_col = string_cols.apply(lambda c: c.str.len().mean()).idxmax()
        print(f"✅ Auto-selected tweet column: '{tweet_col}'")

    def analyze_sentiment_llm(tweet):
        prompt = f"""
You are a sentiment expert. Classify the tweet as one word: Positive, Negative, or Neutral.

Tweet: "{tweet}"
Sentiment:"""
        try:
            res = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.0
            )
            return res['choices'][0]['message']['content'].strip()
        except Exception as e:
            print("⚠️ Error analyzing tweet sentiment:", e)
            return "Error"

    tqdm.pandas()
    df["sentiment"] = df[tweet_col].progress_apply(analyze_sentiment_llm)
    return df


# --- Final Synthesis ---
def synthesize_report(chat_json, screen_json, sentiment_df):
    sentiment_counts = sentiment_df["sentiment"].value_counts().to_dict()
    sentiment_summary = f"Sentiment counts: {sentiment_counts}"

    prompt_template = """
You are a NeuroAI fusion advisor from the future.

Combine these:
1. WhatsApp analysis:
{chat_json}

2. Screen time report:
{screen_json}

3. Twitter sentiment:
{sentiments}

Summarize teen mental health:
- Mood and stress pattern
- Top 3 issues
- Mindfulness movie/song list
- Futuristic habit suggestions

Respond warmly and clearly.
"""
    prompt = PromptTemplate(
        input_variables=["chat_json", "screen_json", "sentiments"],
        template=prompt_template
    )
    formatted_prompt = prompt.format(
        chat_json=json.dumps(chat_json, indent=2) if isinstance(chat_json, dict) else str(chat_json),
        screen_json=json.dumps(screen_json, indent=2) if isinstance(screen_json, dict) else str(screen_json),
        sentiments=sentiment_summary
    )
    return llm(formatted_prompt)  # <-- fixed here


# --- MAIN PIPELINE ---
if __name__ == "__main__":
    whatsapp_file = "whatsapp_chat.txt"
    screen_time_file = "screentime.csv"
    twitter_file = "teen_tweets.csv"

    try:
        # 1. WhatsApp
        chat_msgs = extract_whatsapp_messages(whatsapp_file)
        chat_result = analyze_chat(chat_msgs) if chat_msgs else "No usable messages."

        # 2. Screen Time
        df_screen = load_screen_time(screen_time_file)
        screen_result = analyze_screen_time(df_screen)

        # 3. Twitter Sentiment
        df_tweets = pd.read_csv(twitter_file)
        sentiment_df = analyze_tweets(df_tweets)

        # 4. Synthesis
        final_report = synthesize_report(chat_result, screen_result, sentiment_df)
        print("\n🧠 Final Mental Health Summary:\n", final_report)

    except Exception as e:
        print("❌ Pipeline failed:", e)


OPENAI_API_KEY: None


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-4', '...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
import os
import re
import json
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import openai
from tqdm import tqdm

# --- Load Keys ---
load_dotenv()
print("OPENAI_API_KEY:", os.getenv("OPENAI_API_KEY"))

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)  # No openai_api_key param here



# --- WhatsApp Chat Extraction ---
def extract_whatsapp_messages(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat = f.read()
    # Regex to capture WhatsApp message lines: date, time, sender, message
    pattern = re.compile(r'^\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} [APMapm]{2} - ([^:]+): (.+)$')
    messages = []
    for line in chat.split("\n"):
        match = pattern.match(line)
        if match:
            sender, msg = match.groups()
            if msg.strip() and "media omitted" not in msg.lower():
                messages.append(f"{sender}: {msg}")
    return messages

def analyze_chat(messages, n=50):
    recent = "\n".join(messages[-n:])
    prompt_template = """
You are a futuristic AI therapist from 2030.

Analyze these WhatsApp messages:
- Emotional tone (stress, joy, anxiety)
- Mental clarity & decision style
- Mindset type: proactive, reactive, balanced

Recommend:
- 3 apps/habits to avoid
- 3 uplifting movies and songs
- 3 good daily mental health habits

Output ONLY in JSON format as:
{{"emotional_tone": "...", "clarity": "...", "mindset": "...", "avoid": [...], "recommend": {{"movies": [...], "songs": [...]}}, "habits": [...] }}

Chat:
{chat}
"""
    prompt = PromptTemplate(input_variables=["chat"], template=prompt_template)
    formatted_prompt = prompt.format(chat=recent)

    # Use LangChain's llm.call to get the output text
    response = llm.call_as_llm(formatted_prompt)

    # Try parsing JSON output
    try:
        result = json.loads(response)
        return result
    except json.JSONDecodeError:
        print("⚠️ Failed to parse JSON from chat analysis, raw output:")
        print(response)
        return response

# --- Screen Time Analysis ---
def load_screen_time(csv_path):
    return pd.read_csv(csv_path)

def analyze_screen_time(df):
    readable = df.to_string(index=False)
    prompt_template = """
You are a digital wellness AI.

Analyze this screen time data:
- Focus vs distraction
- Burnout, overuse, addiction
- Decision fatigue signs

Recommend:
- Mental clarity (0-100)
- Avoid apps
- 3 inspiring movies and calming songs
- 3 digital detox habits

Output ONLY in JSON format as:
{{"clarity_score": 0-100, "fatigue": "...", "avoid_apps": [...], "recommend": {{"movies": [...], "songs": [...]}}, "habits": [...] }}

Screen Time Data:
{data}
"""
    prompt = PromptTemplate(input_variables=["data"], template=prompt_template)
    formatted_prompt = prompt.format(data=readable)

    response = llm.call_as_llm(formatted_prompt)

    try:
        result = json.loads(response)
        return result
    except json.JSONDecodeError:
        print("⚠️ Failed to parse JSON from screen time analysis, raw output:")
        print(response)
        return response

# --- Twitter Sentiment Analysis ---
def analyze_tweets(df):
    print("🔍 Columns in CSV:", df.columns.tolist())
    tweet_col = None
    for col in df.columns:
        if col.strip().lower() in ["tweet", "text", "message", "content"]:
            tweet_col = col
            break
    if not tweet_col:
        string_cols = df.select_dtypes(include='object')
        tweet_col = string_cols.apply(lambda c: c.str.len().mean()).idxmax()
        print(f"✅ Auto-selected tweet column: '{tweet_col}'")

    def analyze_sentiment_llm(tweet):
        prompt = f"""
You are a sentiment expert. Classify the tweet as one word: Positive, Negative, or Neutral.

Tweet: "{tweet}"
Sentiment:"""
        try:
            res = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.0
            )
            return res['choices'][0]['message']['content'].strip()
        except Exception as e:
            print("⚠️ Error analyzing tweet sentiment:", e)
            return "Error"

    tqdm.pandas()
    df["sentiment"] = df[tweet_col].progress_apply(analyze_sentiment_llm)
    return df

# --- Final Synthesis ---
def synthesize_report(chat_json, screen_json, sentiment_df):
    sentiment_counts = sentiment_df["sentiment"].value_counts().to_dict()
    sentiment_summary = f"Sentiment counts: {sentiment_counts}"

    prompt_template = """
You are a NeuroAI fusion advisor from the future.

Combine these:
1. WhatsApp analysis:
{chat_json}

2. Screen time report:
{screen_json}

3. Twitter sentiment:
{sentiments}

Summarize teen mental health:
- Mood and stress pattern
- Top 3 issues
- Mindfulness movie/song list
- Futuristic habit suggestions

Respond warmly and clearly.
"""
    prompt = PromptTemplate(
        input_variables=["chat_json", "screen_json", "sentiments"],
        template=prompt_template
    )
    formatted_prompt = prompt.format(
        chat_json=json.dumps(chat_json, indent=2) if isinstance(chat_json, dict) else str(chat_json),
        screen_json=json.dumps(screen_json, indent=2) if isinstance(screen_json, dict) else str(screen_json),
        sentiments=sentiment_summary
    )
    return llm.call_as_llm(formatted_prompt)

# --- MAIN PIPELINE ---
if __name__ == "__main__":
    whatsapp_file = "whatsapp_chat.txt"
    screen_time_file = "screentime.csv"
    twitter_file = "teen_tweets.csv"

    try:
        # 1. WhatsApp
        chat_msgs = extract_whatsapp_messages(whatsapp_file)
        chat_result = analyze_chat(chat_msgs) if chat_msgs else "No usable messages."

        # 2. Screen Time
        df_screen = load_screen_time(screen_time_file)
        screen_result = analyze_screen_time(df_screen)

        # 3. Twitter Sentiment
        df_tweets = pd.read_csv(twitter_file)
        sentiment_df = analyze_tweets(df_tweets)

        # 4. Synthesis
        final_report = synthesize_report(chat_result, screen_result, sentiment_df)
        print("\n🧠 Final Mental Health Summary:\n", final_report)

    except Exception as e:
        print("❌ Pipeline failed:", e)


OPENAI_API_KEY: None


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-4', '...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI  # ✅ updated import
from langchain.prompts import PromptTemplate
import openai
from tqdm import tqdm
import zipfile

# --- Load Keys ---
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4", temperature=0.5, openai_api_key=openai.api_key)  # ✅ updated instantiation


# --- WhatsApp Chat Extraction ---
def extract_whatsapp_messages(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat = f.read()
    pattern = re.compile(r'^\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} [APMapm]{2} - ([^:]+): (.+)$')
    messages = []
    for line in chat.split("\n"):
        match = pattern.match(line)
        if match:
            sender, msg = match.groups()
            if msg.strip() and "media omitted" not in msg.lower():
                messages.append(f"{sender}: {msg}")
    return messages

def analyze_chat(messages, n=50):
    recent = "\n".join(messages[-n:])
    prompt = PromptTemplate(
        input_variables=["chat"],
        template="""
You are a futuristic AI therapist from 2030.

Analyze these WhatsApp messages:
- Emotional tone (stress, joy, anxiety)
- Mental clarity & decision style
- Mindset type: proactive, reactive, balanced

Recommend:
- 3 apps/habits to avoid
- 3 uplifting movies and songs
- 3 good daily mental health habits

Output in JSON:
{{"emotional_tone": "...", "clarity": "...", "mindset": "...", "avoid": [...], "recommend": {{"movies": [...], "songs": [...]}}, "habits": [...] }}

Chat:
{chat}
"""
    )
    return llm.invoke(prompt.format(chat=recent))

# --- Screen Time Analysis ---
def load_screen_time(csv_path):
    return pd.read_csv(csv_path)

def analyze_screen_time(df):
    readable = df.to_string(index=False)
    prompt = PromptTemplate(
        input_variables=["data"],
        template="""
You are a digital wellness AI.

Analyze this screen time data:
- Focus vs distraction
- Burnout, overuse, addiction
- Decision fatigue signs

Recommend:
- Mental clarity (0-100)
- Avoid apps
- 3 inspiring movies and calming songs
- 3 digital detox habits

Output in JSON:
{{"clarity_score": 0-100, "fatigue": "...", "avoid_apps": [...], "recommend": {{"movies": [...], "songs": [...]}}, "habits": [...] }}

Screen Time:
{data}
"""
    )
    return llm.invoke(prompt.format(data=readable))

# --- Twitter Sentiment Analysis ---
def analyze_tweets(df):
    # Try to detect tweet column
    print("🔍 Columns in CSV:", df.columns.tolist())
    tweet_col = None
    for col in df.columns:
        if col.strip().lower() in ["tweet", "text", "message", "content"]:
            tweet_col = col
            break
    if not tweet_col:
        string_cols = df.select_dtypes(include='object')
        tweet_col = string_cols.apply(lambda c: c.str.len().mean()).idxmax()
        print(f"✅ Auto-selected tweet column: '{tweet_col}'")

    def analyze_sentiment_llm(tweet):
        try:
            prompt = f"""
You are a sentiment expert. Classify the tweet as one word: Positive, Negative, or Neutral.

Tweet: "{tweet}"
Sentiment:"""
            res = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.0
            )
            return res['choices'][0]['message']['content'].strip()
        except Exception as e:
            print("⚠️", e)
            return "Error"

    tqdm.pandas()
    df["sentiment"] = df[tweet_col].progress_apply(analyze_sentiment_llm)
    return df

# --- Final Synthesis ---
def synthesize_report(chat_json, screen_json, sentiment_df):
    sentiment_counts = sentiment_df["sentiment"].value_counts().to_dict()
    sentiment_summary = f"Sentiment counts: {sentiment_counts}"

    prompt = PromptTemplate(
        input_variables=["chat_json", "screen_json", "sentiments"],
        template="""
You are a NeuroAI fusion advisor from the future.

Combine these:
1. WhatsApp analysis:
{chat_json}

2. Screen time report:
{screen_json}

3. Twitter sentiment:
{sentiments}

Summarize teen mental health:
- Mood and stress pattern
- Top 3 issues
- Mindfulness movie/song list
- Futuristic habit suggestions

Respond warmly and clearly.
"""
    )
    return llm.invoke(prompt.format(
        chat_json=chat_json,
        screen_json=screen_json,
        sentiments=sentiment_summary
    ))

# --- MAIN PIPELINE ---
if __name__ == "__main__":
    whatsapp_file = "whatsapp_chat.txt"
    screen_time_file = "screentime.csv"
    twitter_file = "teen_tweets.csv"

    try:
        # 1. WhatsApp
        chat_msgs = extract_whatsapp_messages(whatsapp_file)
        chat_result = analyze_chat(chat_msgs) if chat_msgs else "No usable messages."

        # 2. Screen Time
        df_screen = load_screen_time(screen_time_file)
        screen_result = analyze_screen_time(df_screen)

        # 3. Twitter Sentiment
        df_tweets = pd.read_csv(twitter_file)
        sentiment_df = analyze_tweets(df_tweets)

        # 4. Synthesis
        final_report = synthesize_report(chat_result, screen_result, sentiment_df)
        print("\n🧠 Final Mental Health Summary:\n", final_report)

    except Exception as e:
        print("❌ Pipeline failed:", e)


AttributeError: module 'openai' has no attribute 'DefaultHttpxClient'